In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input
import numpy as np
import pandas as pd

batch_size = 64
epochs = 100
latent_dim = 64
num_samples = 127606
filename = 'English_hindi.csv'



In [7]:
# vectorize the data
input_texts = []
target_texts = []
input_characters = []
target_characters = []

df = pd.read_csv(filename)
input_texts = df['English'].values.tolist()
target_texts = df['Hindi'].values.tolist()

for i in range(len(target_texts)):
    target_texts[i] = '\t' + target_texts[i] + '\n'
    for char in input_texts[i]:
        if char not in input_characters:
            input_characters.append(char)
    for char in target_texts[i]:
        if char not in target_characters:
            target_characters.append(char)        

TypeError: can only concatenate str (not "float") to str

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [ ]:
print('i/p characters : {}'.format(len(input_characters)))
print('targert characters : {}'.format(len(target_characters)))
print('encoder characters : {}'.format(num_encoder_tokens))
print('decoder characters : {}'.format(num_decoder_tokens))
print('max encoder : {}'.format(max_encoder_seq_len))
print('max decoder : {}'.format(max_decoder_seq_len))

In [7]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, "'": 4, ',': 5, '-': 6, '.': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'K': 30, 'L': 31, 'M': 32, 'N': 33, 'O': 34, 'P': 35, 'Q': 36, 'R': 37, 'S': 38, 'T': 39, 'U': 40, 'V': 41, 'W': 42, 'Y': 43, 'a': 44, 'b': 45, 'c': 46, 'd': 47, 'e': 48, 'f': 49, 'g': 50, 'h': 51, 'i': 52, 'j': 53, 'k': 54, 'l': 55, 'm': 56, 'n': 57, 'o': 58, 'p': 59, 'q': 60, 'r': 61, 's': 62, 't': 63, 'u': 64, 'v': 65, 'w': 66, 'x': 67, 'y': 68, 'z': 69}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '5': 10, ':': 11, '?': 12, 'I': 13, '|': 14, 'ँ': 15, 'ं': 16, 'ः': 17, 'अ': 18, 'आ': 19, 'इ': 20, 'ई': 21, 'उ': 22, 'ऊ': 23, 'ऋ': 24, 'ए': 25, 'ऐ': 26, 'ऑ': 27, 'ओ': 28, 'औ': 29, 'क': 30, 'ख': 31, 'ग': 32, 'घ': 33, 'च': 34, 'छ': 35, 'ज': 36, 'झ': 37, 'ञ': 38, 'ट': 39, 'ठ': 40, 'ड': 41, 'ढ': 42,

In [8]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')

In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i,t-1,target_token_index[char]] = 1.
    decoder_input_data[i,t+1:,target_token_index[' ']] = 1.
    decoder_target_data[i,t:,target_token_index[' ']] = 1.

In [10]:
encoder_input_data[0].shape

(107, 70)

In [11]:
# Define an input sequence and process it
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states
encoder_states=[state_h,state_c]


In [12]:
# Set up decoder , using ` encoder_states` as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well . we don't use the 
# return states in the training model , but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs= decoder_dense(decoder_outputs)

In [13]:
# Define the model that will turn 
# `encoder_input_data` and `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics= ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size= batch_size, epochs = epochs , 
           validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 29s 691ms/step - loss: 1.7165 - accuracy: 0.7140 - val_loss: 1.5895 - val_accuracy: 0.6897
Epoch 2/100
35/35 [==============================] - 23s 658ms/step - loss: 0.8669 - accuracy: 0.8065 - val_loss: 1.5141 - val_accuracy: 0.6912
Epoch 3/100
35/35 [==============================] - 23s 657ms/step - loss: 0.8318 - accuracy: 0.8100 - val_loss: 1.4394 - val_accuracy: 0.6912
Epoch 4/100
35/35 [==============================] - 23s 657ms/step - loss: 0.7905 - accuracy: 0.8121 - val_loss: 1.7417 - val_accuracy: 0.6927
Epoch 5/100
35/35 [==============================] - 23s 661ms/step - loss: 0.7513 - accuracy: 0.8149 - val_loss: 1.2340 - val_accuracy: 0.7014
Epoch 6/100
35/35 [==============================] - 23s 652ms/step - loss: 0.7081 - accuracy: 0.8260 - val_loss: 1.1587 - val_accuracy: 0.7132
Epoch 7/100
35/35 [==============================] - 23s 657ms/step - loss: 0.6526 - accuracy: 0.8366 - val_loss: 1.0843 - val_accuracy:

In [21]:
# Inference setup
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_inputs_h = Input(shape=(latent_dim, ))
decoder_state_inputs_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_inputs_h, decoder_state_inputs_c]
decoder_outputs , state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs]+decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [24]:
# Inference loop
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value=encoder_model.predict(input_seq)
    
    #Generate empty target sequence length 1 
    target_seq = np.zeros((1,1, num_decoder_tokens))
    # Populate the 1st character of target sequence with start character 
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    #Sampling loop for a batch of sequences 
    # (to samplify, here we assume a batch of size 1 )
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        #sample a token
        sampled_token_index= np.argmax(output_tokens[0,-1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        #Exit condition : either hit max length
        #or find stop character.
        if(sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True
        
        #Update the target sequence (of length 1)
        target_seq = np.zeros((1,1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        
        #Update states
        states_value = [h, c]
    return decoded_sentence
        

In [25]:
for seq_index in range(20):
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print(input_texts[seq_index])
    print(decoded_sentence)

-
Help!
उसकी बार की ज़ियलग गा।

-
Jump.
मुझे कल आए ही कि मेट ग़लन मालूूँगा।

-
Jump.
मुझे कल आए ही कि मेट ग़लन मालूूँगा।

-
Jump.
मुझे कल आए ही कि मेट ग़लन मालूूँगा।

-
Hello!
उसकी बार की ज़ियलग गा।

-
Hello!
उसकी बार की ज़ियलग गा।

-
Cheers!
मुझे कल आए ही कि मेट ग़लन माले जा अक्छा लग ला।

-
Cheers!
मुझे कल आए ही कि मेट ग़लन माले जा अक्छा लग ला।

-
Got it?
मुझे कल आए ही कि मही आँगा।

-
I'm OK.
मैं तुम्हें यह बहुत नहीं हूँ।

-
Awesome!
मुझे कल आए ही कि मेट ग़लन माले जा अक्छा लग ला।

-
Come in.
मुझे कल बात किया हो हैं।

-
Get out!
मुझे कल आए ही कि मही आँगा।

-
Go away!
मुझे कल आए ही कि मही आँगा।

-
Goodbye!
मुझे कल आए ही कि मेट ग़लन माले जा अक्छा को लिए बहुत करता है।

-
Perfect!
मुझे उसके प्रस्ताव को अच्छा कर लड़ा था।

-
Perfect!
मुझे उसके प्रस्ताव को अच्छा कर लड़ा था।

-
Welcome.
तुम कितने साल के हो?

-
Welcome.
तुम कितने साल के हो?

-
Have fun.
मेरे पापा अभी अभी घर वापस आएँ हैं।



In [30]:
cd = pd.read_csv('English_hindi.csv')
cd.shape

(127607, 3)